In [28]:
pip install deep-translator yandex-translater

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from deep_translator import GoogleTranslator
import random
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from yandex.Translater import Translater

In [12]:
df = pd.read_csv('raw_data.csv') # изменить путь
df

,idx,text,label
0,10026,[PERSON]\n\nУважаемый [PERSON]!\n\n[ORGANIZATI...,Блок директора по проектированию
1,1005,[ORGANIZATION] инжиниринг общество с ограничен...,Блок деректора по газу
2,1010,[ORGANIZATION] ИНВЕСТ Общество с ограниченной ...,Блок заместителя генерального директора по без...
3,1013,[ORGANIZATION] филиал [OBJECT] имени [PERSON] ...,Блок технического директора
4,10186,[ORGANIZATION] ЗАМЕСТИТЕЛЬ ГЕНЕРАЛЬНОГО ДИРЕКТ...,Блок директора по проектированию
...,...,...,...
1416,7295,[ORGANIZATION] юридический адрес: [LOCATION] О...,Блок технического директора
1417,7296,[ORGANIZATION] сообщает [PERSON] о готовности ...,Блок технического директора
1418,7297,[ORGANIZATION] Юридический адрес: [LOCATION] О...,Блок технического директора
1419,27278G,ООО [НАЗВАНИЕ_КОМПАНИИ] Контактная информация:...,Проект «Трубопроводный транспорт Ещё одного НГКМ»


In [ ]:
# tr = Translater()

def translate(count: int, base_idx: str, base_text: str, label: str) -> list:
    results = []
    
    # Если длина текса больше 4000 символов - разбиваем на куски
    if len(base_text) > 3000:
        sub_str_count = (len(base_text) + 2999) // 3000
        new_texts = ['' for _ in range(count)]
        exploded_text = [base_text[i:i + 3000] for i in range(0, len(base_text), 3000)]
        for i in range(sub_str_count-1):
            while exploded_text[i][-1] != ' ':
                exploded_text[i+1] = exploded_text[i][-1] + exploded_text[i+1]
                exploded_text[i] = exploded_text[i][:-1]
        for i in range(sub_str_count):
            new_sub_strs = [i['text'] for i in translate(count, base_idx, exploded_text[i], label)]
            for j in range(count):
                new_texts[j] += new_sub_strs[j]
    
        for i in range(count):
            results.append({
                'idx': f'new_{label}_translate_{base_idx}_{i+1}',
                'text': new_texts[i],
                'label': label
            })
            
        return results
        
    
    # список языков для случайного перевода (можно менять)
    languages = ['en', 'de', 'fr', 'es', 'it']
    for i in range(count):
        #new_text = '' + f" [перевод_{i+1}]" # base_text + f" [перевод_{i+1}]" # текст-рыба

        try:
            # выбираем случайный язык для “обратного перевода”
            lang1 = random.choice(languages)
            lang2 = random.choice(languages)

            # переводим с русского на lang1
            text_foreign = GoogleTranslator(source='ru', target=lang1).translate(base_text)
            # переводим с lang1 на lang2
            new_text = GoogleTranslator(source=lang1, target='ru').translate(text_foreign)

            # и обратно на русский
            new_text = GoogleTranslator(source=lang2, target='ru').translate(text_foreign)

            # лёгкий sleep, чтобы не словить лимиты API
            # new_text = yt.translate('ru', lang1, base_text)
            # new_text = yt.translate(lang1, lang2, new_text)
            # new_text = yt.translate(lang2, 'ru', new_text)
            
            time.sleep(0.3)

        except Exception as e:
            print(base_text)
            print(f"⚠️ Ошибка перевода письма {base_idx}_{i+1}: {e}")
            # если не удалось — fallback: просто тег
            new_text = base_text + f" [перевод_{i+1}]"

        results.append({
            'idx': f'new_{label}_translate_{base_idx}_{i+1}',
            'text': new_text,
            'label': label
        })
    return results

In [27]:
# df[df['text'].str.len() > 10000]

# text = df[df['idx'] == '1215']['text'].values[0]
# text
text = '__1__ Филиал __2__ Исполнительному директору Проекта __1__ А.А. [PERSON] тер. [OBJECT] [OBJECT], панель V, д. 4, [LOCATION] тел.: [CONTACT], факс: [CONTACT] e-mail: [CONTACT], www.[DOMAIN] ОКПО [ID], ОГРН [DOCUMENT_NUMBER], ИНН [ID], КПП [ID] О неудовлетворительном состоянии проездов на [OBJECT] Уважаемый [PERSON]! На основании поручения Председателя Правления [ORGANIZATION] от [DATE_TIME]'


translate(1, '1215', text, 'Имущественные вопросы')

__1__ Филиал __2__ Исполнительному директору Проекта __1__ А.А. [PERSON] тер. [OBJECT] [OBJECT], панель V, д. 4, [LOCATION] тел.: [CONTACT], факс: [CONTACT] e-mail: [CONTACT], www.[DOMAIN] ОКПО [ID], ОГРН [DOCUMENT_NUMBER], ИНН [ID], КПП [ID] О неудовлетворительном состоянии проездов на [OBJECT] Уважаемый [PERSON]! На основании поручения Председателя Правления [ORGANIZATION] от [DATE_TIME]
⚠️ Ошибка перевода письма 1215_1: blocked or not found


[{'idx': 'new_Имущественные вопросы_translate_1215_1',
  'text': '__1__ Филиал __2__ Исполнительному директору Проекта __1__ А.А. [PERSON] тер. [OBJECT] [OBJECT], панель V, д. 4, [LOCATION] тел.: [CONTACT], факс: [CONTACT] e-mail: [CONTACT], www.[DOMAIN] ОКПО [ID], ОГРН [DOCUMENT_NUMBER], ИНН [ID], КПП [ID] О неудовлетворительном состоянии проездов на [OBJECT] Уважаемый [PERSON]! На основании поручения Председателя Правления [ORGANIZATION] от [DATE_TIME] [перевод_1]',
  'label': 'Имущественные вопросы'}]

In [16]:
a = 'faknksbdfdsv,msdvs.dgs.dfg'
# print([a[i:i + 5] for i in range(0, len(a), 5)])
a[:-1]

'faknksbdfdsv,msdvs.dgs.df'

In [17]:
[ОРГАНИЗАЦИЯ] ПРИКАЗ [ДАТА_ВРЕМЯ] № [НОМЕР_ДОКУМЕНТА] об утверждении Положения о непрофильных активах и порядке отчуждения непрофильных активов. В целях приведения локальных положений Группы [ОРГАНИЗАЦИЯ] по управлению непрофильными активами в соответствие с положениями [ОРГАНИЗАЦИИ] ПРИКАЗЫВАЮ: 1. Утвердить Положение о Комиссии по непрофильным активам [ОРГАНИЗАЦИИ] и организаций Группы компаний [ОРГАНИЗАЦИЯ] в новой редакции (Приложение 1). 2. Утвердить Положение о порядке отчуждения непрофильного имущества [ОРГАНИЗАЦИИ] и организаций группы компаний [ОРГАНИЗАЦИЯ] в новой редакции (приложение 2). 3. Утвердить новый состав Комиссии по непрофильным активам [ОРГАНИЗАЦИЯ] (Приложение 3). 4. Признайте команды [ОРГАНИЗАЦИЯ] для [ДАТА_ВРЕМЯ], а не для [НОМЕР_ДОКУМЕНТА], и для [ДАТА_ВРЕМЯ], а не для [НОМЕР_ДОКУМЕНТА], как недействительные. 5. Распространить действие настоящего Приказа на организации группы компаний [ОРГАНИЗАЦИЯ], в которых компания имеет право прямо или косвенно распоряжаться более чем 50 голосами (приложение 4). 6. Поручить контроль за исполнением поручения заместителю председателя совета директоров [ЛИЦО] председателю совета директоров [ЛИЦО] Приложение 1 к приказу [ОРГАНИЗАЦИЯ] от [ДАТА_ВРЕМЯ] № [НОМЕР_ДОКУМЕНТА] ПОЛОЖЕНИЕ О КОМИССИИ [ОРГАНИЗАЦИЯ] И ОРГАНИЗАЦИЯХ ГРУППЫ КОМПАНИЙ [ОРГАНИЗАЦИЯ] ДЛЯ НЕСУЩЕСТВЕННЫЕ АКТИВЫ 1. Общие положения Настоящее Положение определяет цели, задачи, организацию деятельности Комиссии [ОРГАНИЗАЦИИ] по непрофильным активам (далее – Комиссия), права и обязанности Председателя, Секретаря и членов Комиссии, а также порядок представления отчетов о ходе ее работы. Требования настоящего Положения обязательны для применения при отчуждении непрофильных активов [ОРГАНИЗАЦИЯ] и организаций группы компаний [ОРГАНИЗАЦИЯ]. Дочерние общества и организации [ОРГАНИЗАЦИЯ] обеспечивают утверждение локальных нормативных актов (положений), регулирующих в соответствии с настоящим Положением порядок работы комиссий по непрофильным активам в дочерних обществах, а также в объектах инвестирования дочерних обществ, являющихся организациями Группы компаний [ОРГАНИЗАЦИЯ]. Представители интересов [ОРГАНИЗАЦИИ] и дочерних обществ [ОРГАНИЗАЦИИ] в органах управления организаций группы компаний [ОРГАНИЗАЦИЯ] обеспечивают на основании настоящего Положения утверждение локальных нормативных актов (положений), регулирующих порядок работы комиссий по непрофильным активам в этих организациях группы компаний [ОРГАНИЗАЦИЯ]. 2. Основные цели и задачи Комиссии 2.1. Комиссия создается для обеспечения реализации Стратегии управления имуществом и иными активами [ОРГАНИЗАЦИЯ], Программы отчуждения неосновных активов [ОРГАНИЗАЦИЯ], утвержденной [ОРГАНИЗАЦИЯ], и Положения о Комиссии [ОРГАНИЗАЦИЯ] по неосновным активам. Комиссия в своей деятельности руководствуется законодательством Российской Федерации, решениями органов управления [ОРГАНИЗАЦИЯ] и [ОРГАНИЗАЦИЯ], иными внутренними документами, в том числе Положением о Порядкеотчуждения непрофильных активов [ОРГАНИЗАЦИЯ] и организаций Группы Газпром, Положения о комиссии [ОРГАНИЗАЦИЯ] по непрофильным активам, Положения о порядке отчуждения непрофильных активов [ОРГАНИЗАЦИЯ] и организаций Группы компаний [ОРГАНИЗАЦИЯ] и настоящего Положения. 2.2. Основной задачей Комиссии является принятие решений об отнесении активов, принадлежащих [ОРГАНИЗАЦИЯ] и организациям Группы компаний [ОРГАНИЗАЦИЯ], к непрофильным, а также определение порядка их реализации. В отношении активов, принадлежащих [ОРГАНИЗАЦИИ] и организациям Группы компаний [ОРГАНИЗАЦИЯ], если их балансовая (остаточная) стоимость по данным бухгалтерского учета, либо рыночная стоимость, либо кадастровая стоимость (для земельных участков)1 составляет 100 (Сто) миллионов и более рублей, включая НДС, решения принимаются Комиссией [ОРГАНИЗАЦИИ] по непрофильному имуществу. В отношении активов, находящихся в собственности [ОРГАНИЗАЦИЯ], а также организаций Группы компаний [ОРГАНИЗАЦИЯ], если их балансовая (остаточная) стоимость по данным бухгалтерского учета, либо рыночная стоимость, либо кадастровая стоимость (для земельных участков)1 составляет от 30 до 100 миллионов рублей, включая НДС, решения принимаются Комиссией [ОРГАНИЗАЦИЯ]. В отношении активов, находящихся в собственности организаций группы компаний [ОРГАНИЗАЦИЯ], если их балансовая (остаточная) стоимость по данным бухгалтерского учета, либо рыночная стоимость, либо кадастровая стоимость (для земельных участков)1 составляет менее 30 миллионов рублей с учетом НДС, решения принимаются комиссией Организации группы компаний [ОРГАНИЗАЦИЯ]. 2.3. Для выполнения возложенной на нее задачи Комиссия определяет критерии отнесения активов к непрофильным, а также принимает следующие решения: об отнесении активов к непрофильным; о признании непрофильных активов подлежащими/не подлежащими отчуждению; о целесообразности использования отдельных способов отчуждения непрофильных активов; об определении условий отчуждения непрофильных активов2; о необходимости предпродажной подготовки непрофильных активов, включая оценку их рыночной стоимости. 3. Организация деятельности Комиссии 3.1. Персональный состав Комиссии утверждается приказом [ОРГАНИЗАЦИЯ]. 3.2. Вопросы отнесения активов к непрофильным выносятся на рассмотрение Комиссии по инициативе органов управления [ОРГАНИЗАЦИИ], заместителя Председателя Правления и руководителей подразделений, непосредственно подчиненных Председателю Правления [ОРГАНИЗАЦИИ], структурных подразделений [ОРГАНИЗАЦИИ]. 3.3. Организация Группы компаний [ОРГАНИЗАЦИЯ] перед представлением материалов на рассмотрение Комиссии согласовывает обоснование возможности отчуждения актива, а также предлагаемый способ его отчуждения со структурными подразделениями [ОРГАНИЗАЦИЯ], ответственными за контроль эффективности долгосрочных финансовых вложений [ОРГАНИЗАЦИЯ] в соответствующие дочерние общества [ОРГАНИЗАЦИЯ],которыйЭта организация Группы компаний [ОРГАНИЗАЦИОН] (кроме того, подразделение, ответственное за [ОРГАНИЗАЦИОН]) находится под контролем. Предоставление материалов на рассмотрение Комиссии касается дочерних компаний [ОРГАНИЗАЦИОН], в том числе в отношении организаций, контролируемых Группой компаний [ОРГАНИЗАЦИОН], что является ее вышеуказанными объектами инверсии, или подразделением, ответственным за [ОРГАНИЗАЦИОН]. 3.4 Комиссия в областях своей компетенции: запрашивает у структурных подразделений [ОРГАНИЗАЦИОН] и организаций Группы компаний [ОРГАНИЗАЦИОН] документы, материалы и информацию, необходимые для ее деятельности; установить для структурных подразделений [ORGANIZACIÓN] и организаций Группы компаний [ORGANIZACIÓN] условия и форму представления 1. В отношении объектов недвижимости, имеющих следующие признаки идентификации (принадлежность, окончательность и другие характеристики), как если бы они были приобретены в пользу покупателя или включены в единую группу недвижимого имущества, общая стоимость балансовой (остаточной) либо товара, либо имущества (для земли) рассчитывается. 2. Во избежание дублирования стандартов и правил работы Комиссии и Инверсионных комитетов Общества вопросы удобства осуществления дополнительных видов деятельности могут быть приняты к рассмотрению Комитетом только после их рассмотрения Инверсионным комитетом, соответствующим запрашиваемым документам, материалам и информации. 3.5. Заседания Комиссии проводятся раздельно по мере необходимости. Оно действительно для заседания Комиссии с участием всех членов от общего числа наших членов. При проведении заседания Комиссии в официальном присутствии решения принимаются в ходе голосования простым большинством голосов членов Комиссии, присутствующих на заседании. Когда заседание Комиссии отмечается благоприятно, голосование будет проводиться с помощью бумаг, которым наши члены могут позавидовать в соответствии с настоящим Регламентом. В бюллетенях для голосования должна быть указана продолжительность представления полного документа. В случае неправильного голосования решающим голосом является Председатель Комиссии и одновременно Вице-президент Комиссии. 3.6. Распорядок дня заседания Комиссии, письмо о созыве заседания с указанием формы, времени, времени и места вашего торжества, Firmada por el Presidente de la Commission (под его эгидой - Вице-президент Комиссии), информационные материалы и бюллетени для голосования (если заседание проводится очно). ausencia) отправить в лос-мимброс-де-ла-Комиссию за 5 естественных дней до окончания заседания Комиссии. В исключительных случаях, при необходимости принятия оперативных решений, это место может быть сокращено по решению Председателя Комиссии (по его поручению - Вице-президента Комиссии). 3.7. Результаты голосования членов Комитета оформляются актами в течение 3-х дней после окончания заседания. Протокол утверждается Президентом Комиссии (по его поручению - Вице-президентом Комиссии) и Секретарем Комиссии. После принятия Комитетомрешения заочным голосованием протокол заседания составляется в течение 3 календарных дней, следующих за датой, установленной для вручения заполненных бюллетеней. К протоколу прилагаются бюллетени для голосования, подписанные членами Комиссии. Бюллетень члена Комиссии, не представленный Секретарю Комиссии в срок, установленный для голосования, либо неправильно заполненный, не будет учитываться при подведении итогов голосования. Члены Комиссии, проголосовавшие против решения, а также воздержавшиеся, имеют право выразить свое особое мнение в письменной форме, которое приобщается к протоколу заседания Комиссии. Протокол заседания Комиссии должен содержать сведения о членах Комиссии, голосовавших по вопросам повестки дня заседания, результатах голосования, а также решениях, принятых на заседании. 3.8. Решение Комиссии (выписки из протокола заседания Комиссии) направляется для исполнения в соответствующее структурное подразделение или филиал [ОРГАНИЗАЦИИ] в течение 3 рабочих дней со дня оформления соответствующего протокола Комиссии. 4. Права и обязанности Президента, Секретаря и членов Комиссии 4.1. Руководитель структурного подразделения [ОРГАНИЗАЦИЯ] назначается председателем Комиссии, одной из основных задач которой является обеспечение управления взаимодополняющими активами [ОРГАНИЗАЦИИ] и организаций Группы компаний [ОРГАНИЗАЦИЯ] (далее – Департамент по правовым, корпоративным и имущественным вопросам). В случае отсутствия Председателя Комиссии функции Президента выполняет его заместитель. Председатель Комиссии: организует работу Комиссии и выполнение возложенных на нее задач; созывает, проводит заседания Комиссии и председательствует на них; обеспечивает коллегиальное обсуждение рассматриваемых вопросов; при необходимости дает указания членам Комиссии;

[ORGANIZATION] ПРИКАЗ [DATE_TIME] No [DOCUMENT_NUMBER] Об утверждении Положений по непрофильным активам и порядке отчуждения непрофильных активов В целях приведения локальных нормативных актов Группы [ORGANIZATION] по вопросам управления непрофильными активами в соответствии с нормативными актами [ORGANIZATION] ПРИКАЗЫВАЮ: 1. Утвердить Положение о Комиссии по непрофильным активам [ORGANIZATION] и организаций Группы компаний [ORGANIZATION] в новой редакции (Приложение 1). 2. Утвердить Положение о порядке отчуждения непрофильных активов [ORGANIZATION] и организаций Группы компаний [ORGANIZATION] в новой редакции (Приложение 2). 3. Утвердить состав Комиссии по непрофильным активам [ORGANIZATION] в новом составе (Приложение 3). 4. Признать утратившими силу приказы [ORGANIZATION] от [DATE_TIME] No [DOCUMENT_NUMBER] и от [DATE_TIME] No [DOCUMENT_NUMBER]. 5. Распространить действие настоящего Приказа на организации Группы компаний [ORGANIZATION], в которых Компания имеет право прямо или косвенно распоряжаться более 50 голосов (Приложение 4). 6. Контроль исполнения приказа возложить на заместителя Председателя Правления [PERSON] Председатель Правления [PERSON] Приложение 1 к приказу [ORGANIZATION] от [DATE_TIME] No [DOCUMENT_NUMBER] ПОЛОЖЕНИЕ О КОМИССИИ [ORGANIZATION] И ОРГАНИЗАЦИЙ ГРУППЫ КОМПАНИЙ [ORGANIZATION] ПО НЕПРОФИЛЬНЫМ АКТИВАМ 1. Общие положения Настоящее Положение определяет цели, задачи, организацию деятельности Комиссии [ORGANIZATION] по непрофильным активам (далее – Комиссия), права и обязанности Председателя, Секретаря и членов Комиссии, а также порядок представления отчетов о ходе ее работы. Требования настоящего Положения обязательны для применения при отчуждении непрофильных активов [ORGANIZATION] и организаций Группы компаний [ORGANIZATION]. Дочерние общества и организации [ORGANIZATION] обеспечивают утверждение локальных нормативных актов (положений), регламентирующих в соответствии с настоящим Положением порядок работы комиссий по непрофильным активам в дочерних обществах, а также в объектах вложений дочерних обществ, являющихся организациями Группы компаний [ORGANIZATION]. Представители интересов [ORGANIZATION] и дочерних обществ [ORGANIZATION] в органах управления организаций Группы компаний [ORGANIZATION] обеспечивают на основе настоящего Положения утверждение локальных нормативных актов (положений), регламентирующих порядок работы комиссий по непрофильным активам в таких организациях Группы компаний [ORGANIZATION]. 2. Основные цели и задачи Комиссии 2.1. Комиссия создается в целях обеспечения реализации Стратегии по управлению имуществом и иными активами [ORGANIZATION], Программы отчуждения непрофильных активов [ORGANIZATION], утверждаемой [ORGANIZATION], и Положения о комиссии [ORGANIZATION] по непрофильным активам. Комиссия в своей деятельности руководствуется законодательством Российской Федерации, решениями органов управления [ORGANIZATION] и [ORGANIZATION], иными внутренними документами, в том числе Положением о порядке отчуждения непрофильных активов [ORGANIZATION] и организаций Группы Газпром, Положением о комиссии [ORGANIZATION] по непрофильным активам, Положением о порядке отчуждения непрофильных активов [ORGANIZATION] и организаций Группы компаний [ORGANIZATION] и настоящим Положением. 2.2. Основной задачей Комиссии является принятие решений об отнесении активов, принадлежащих [ORGANIZATION] и организациям Группы компаний [ORGANIZATION], к категории непрофильных, а также определение порядка их реализации. В отношении активов, принадлежащих [ORGANIZATION] и организациям Группы компаний [ORGANIZATION], в случае, если их балансовая (остаточная) стоимость по данным бухгалтерского учета, или рыночная стоимость, или кадастровая стоимость (для земельных участков)1 составляет 100 (сто) и более миллионов рублей с учетом НДС решения принимает Комиссия [ORGANIZATION] по непрофильным активам. В отношении активов, принадлежащих [ORGANIZATION], а также организациям Группы компаний [ORGANIZATION] в случае, если их балансовая (остаточная) стоимость по данным бухгалтерского учета, или рыночная стоимость, или кадастровая стоимость (для земельных участков)1 составляет от 30 до 100 миллионов рублей с учетом НДС, решения принимаются Комиссией [ORGANIZATION]. В отношении активов, принадлежащих организациям Группы компаний [ORGANIZATION], в случае, если их балансовая (остаточная) стоимость по данным бухгалтерского учета, или рыночная стоимость, или кадастровая стоимость (для земельных участков)1 составляет менее 30 миллионов рублей с учетом НДС, решения принимаются Комиссией организации Группы компаний [ORGANIZATION]. 2.3. Для выполнения возложенной на нее задачи, Комиссия определяет критерии отнесения активов к категории непрофильных, а также принимает следующие решения: об отнесении активов к категории непрофильных; о признании непрофильных активов подлежащими/не подлежащими отчуждению; о целесообразности применения тех или иных способов отчуждения непрофильных активов; об определении условий отчуждения непрофильных активов2; о необходимости проведения предпродажной подготовки непрофильных активов, в том числе оценки их рыночной стоимости. 3. Организация деятельности Комиссии 3.1. Персональный состав Комиссии утверждается приказом [ORGANIZATION]. 3.2. Вопросы об отнесении активов к категории непрофильных вносятся на рассмотрение Комиссии по инициативе органов управления [ORGANIZATION], заместителей Председателя Правления и руководителей подразделений прямого подчинения Председателю Правления [ORGANIZATION], структурных подразделений [ORGANIZATION]. 3.3. Организация Группы компаний [ORGANIZATION] перед представлением материалов на рассмотрение Комиссии согласовывает обоснование целесообразности отчуждения актива, а также предполагаемого способа его отчуждения со структурными подразделениями [ORGANIZATION], ответственными за осуществление контроля за обеспечением эффективности долгосрочных финансовых вложений [ORGANIZATION] в соответствующие дочерние общества [ORGANIZATION], которым подконтрольна эта организация Группы компаний [ORGANIZATION] (далее - ответственное подразделение [ORGANIZATION]). Представление материалов на рассмотрение Комиссии осуществляется дочерними обществами [ORGANIZATION], в том числе в отношении подконтрольных организаций Группы компаний [ORGANIZATION], являющихся их объектами вложений, либо ответственным подразделением [ORGANIZATION]. 3.4 Комиссия по вопросам, входящим в ее компетенцию, имеет право: запрашивать у структурных подразделений [ORGANIZATION] и организаций Группы компаний [ORGANIZATION] необходимые для ее деятельности документы, материалы и информацию; устанавливать для структурных подразделений [ORGANIZATION] и организаций Группы компаний [ORGANIZATION] сроки и форму представления 1 В отношении объектов имущества, имеющих одни и те же идентификационные признаки (место нахождения, назначение и другие признаки), а также отчуждаемых в пользу одного приобретателя либо входящих в единый имущественный комплекс, рассчитывается суммарная балансовая (остаточная) или рыночная или кадастровая стоимость имущества (для земельных участков). 2 Во избежание дублирования норм и правил работы Комиссии и Инвестиционных комитетов Компании, вопросы о целесообразности отчуждения непрофильных активов могут быть приняты к рассмотрению Комиссией только после их рассмотрения на соответствующем Инвестиционном Комитете запрашиваемых документов, материалов и информации. 3.5. Заседания Комиссии проводятся по мере необходимости. Заседание Комиссии является правомочным при участии в нем не менее половины от общего числа ее членов. При очной форме проведения заседания Комиссии решения принимаются открытым голосованием простым большинством голосов членов Комиссии, присутствующих на заседании. При заочной форме проведения заседания Комиссии голосование осуществляется посредством бюллетеней, которые направляются ее членам в соответствии с настоящим Положением. Бюллетени для голосования должны содержать указание на дату представления заполненного бюллетеня. В случае равенства голосов решающим является голос Председателя Комиссии, а в его отсутствие заместителя Председателя Комиссии. 3.6. Повестка заседания Комиссии, письмо о созыве заседания с указанием формы, даты, времени и места его проведения, подписанные Председателем Комиссии (в его отсутствие заместителем Председателя Комиссии), информационные материалы и бюллетени для голосования (при проведении заседания в заочной форме) направляются членам Комиссии за 5 календарных дней до даты проведения заседания Комиссии. В исключительных случаях при необходимости принятия оперативных решений указанный срок может быть сокращен по решению Председателя Комиссии (в его отсутствие заместителя Председателя Комиссии). 3.7. Итоги голосования членов Комиссии оформляются протоколом в течение 3 календарных дней с даты проведения заседания. Протокол подписывается Председателем Комиссии (в его отсутствие заместителем Председателя Комиссии) и Секретарем Комиссии. При принятии Комиссией решений заочным голосованием протокол заседания оформляется в течение 3 календарных дней с даты, установленной для представления заполненных бюллетеней. К протоколу прилагаются подписанные членами Комиссии бюллетени для голосования. Бюллетень члена Комиссии, не представленный Секретарю Комиссии в установленный для голосования срок, либо заполненный ненадлежащим образом, при подведении итогов голосования не учитывается. Члены Комиссии, голосовавшие против принятого решения, а также воздержавшиеся вправе в письменной форме изложить свое особое мнение, которое приобщается к протоколу заседания Комиссии. Протокол заседания Комиссии должен содержать информацию о членах Комиссии, проголосовавших по вопросам повестки заседания, результатах голосования, а также о принятых на заседании решениях. 3.8. Решение Комиссии (выписки из протокола заседания Комиссии) направляются для исполнения в соответствующее структурное подразделение или дочернее общество [ORGANIZATION] в течение 3 рабочих дней с даты оформления соответствующего протокола Комиссии. 4. Права и обязанности Председателя, Секретаря и членов Комиссии 4.1. Председателем Комиссии назначается руководитель структурного подразделения [ORGANIZATION], одной из основных задач которого является обеспечение управления непрофильными активами [ORGANIZATION] и организаций Группы компаний [ORGANIZATION] (далее Департамент по правовым, корпоративным и имущественным вопросам). В отсутствие Председателя Комиссии обязанности Председателя исполняет его заместитель. Председатель Комиссии: организует работу Комиссии и выполнение возложенных на нее задач; созывает, проводит заседания Комиссии и председательствует на них; обеспечивает коллегиальное обсуждение рассматриваемых вопросов; при необходимости дает поручения членам Комиссии;

SyntaxError: invalid character '№' (U+2116) (2596776757.py, line 1)